# K리그 관중 수 예측 머신러닝 모델링

## 목표
K리그 경기 관중 수를 예측하는 머신러닝 모델을 개발하고 성능을 평가합니다.

## 사용 데이터
- K리그1, K리그2 경기 데이터 (2023-2025)
- 날씨, 순위, 팀, 경기장 정보 포함

## 0. 라이브러리 및 데이터 준비

In [11]:
import pandas as pd
import sqlite3
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

# 데이터베이스 경로
DB_PATH = "..\\data\\match.db"

# 1. 데이터 준비 및 특성 공학

In [12]:
# K리그 전체 데이터 로드
with sqlite3.connect(DB_PATH) as conn:
    query_ml = """
        SELECT 
            Meet_Year,
            LEAGUE_NAME,
            Round,
            Game_Datetime,
            Day,
            HomeTeam,
            AwayTeam,
            HomeRank,
            AwayRank,
            Field_Name,
            Audience_Qty,
            Weather,
            Temperature,
            Humidity
        FROM kleague
        WHERE Temperature IS NOT NULL 
          AND Humidity IS NOT NULL
          AND Weather IS NOT NULL
          AND HomeRank IS NOT NULL
          AND HomeRank > 0
          AND AwayRank IS NOT NULL
          AND AwayRank > 0
    """
    df_ml = pd.read_sql(query_ml, conn)

print(f"총 데이터 수: {len(df_ml):,}건")
print(f"\n데이터 기간: {df_ml['Meet_Year'].min()} ~ {df_ml['Meet_Year'].max()}")
print(f"\n컬럼 정보:")
display(df_ml.dtypes.to_frame(name='dtype'))
print(f"\n결측치 확인:")
display(df_ml.isnull().sum())

총 데이터 수: 1,392건

데이터 기간: 2023 ~ 2025

컬럼 정보:


,dtype
Meet_Year,int64
LEAGUE_NAME,object
Round,int64
Game_Datetime,object
Day,object
HomeTeam,object
AwayTeam,object
HomeRank,int64
AwayRank,int64
Field_Name,object



결측치 확인:


Meet_Year        0
LEAGUE_NAME      0
Round            0
Game_Datetime    0
Day              0
HomeTeam         0
AwayTeam         0
HomeRank         0
AwayRank         0
Field_Name       0
Audience_Qty     0
Weather          0
Temperature      0
Humidity         0
dtype: int64

In [13]:
# 특성 공학 (Feature Engineering)
df_ml_fe = df_ml.copy()

# 1. 시즌 단계 특성
def get_season_stage(round_num, league):
    max_round = 38 if league == 'K리그1' else 44
    if round_num <= max_round / 3:
        return 0  # 초반
    elif round_num <= 2 * max_round / 3:
        return 1  # 중반
    else:
        return 2  # 후반

df_ml_fe['Season_Stage'] = df_ml_fe.apply(
    lambda row: get_season_stage(row['Round'], row['LEAGUE_NAME']), axis=1
)

# 2. 주말 여부
df_ml_fe['Is_Weekend'] = df_ml_fe['Day'].isin(['토', '일']).astype(int)

# 3. 순위 차이 (홈팀이 원정팀보다 순위가 높으면 음수)
df_ml_fe['Rank_Difference'] = df_ml_fe['HomeRank'] - df_ml_fe['AwayRank']

# 4. 상위권 대결 여부 (둘 다 5위 이내)
df_ml_fe['Top_Match'] = ((df_ml_fe['HomeRank'] <= 5) & (df_ml_fe['AwayRank'] <= 5)).astype(int)

# 5. 하위권 대결 여부 (둘 다 9위 이하 - K리그1 기준)
df_ml_fe['Bottom_Match'] = ((df_ml_fe['HomeRank'] >= 9) & (df_ml_fe['AwayRank'] >= 9)).astype(int)

# 6. 날씨를 범주로 단순화
def simplify_weather(weather):
    if '맑' in weather or '청' in weather:
        return 'clear'
    elif '비' in weather or '우' in weather:
        return 'rainy'
    elif '눈' in weather or '설' in weather:
        return 'snowy'
    else:
        return 'cloudy'

df_ml_fe['Weather_Simple'] = df_ml_fe['Weather'].apply(simplify_weather)

# 7. 온도 구간
def temp_category(temp):
    if temp < 10:
        return 0  # 추운 날씨
    elif temp < 20:
        return 1  # 선선한 날씨
    elif temp < 28:
        return 2  # 적당한 날씨
    else:
        return 3  # 더운 날씨

df_ml_fe['Temp_Category'] = df_ml_fe['Temperature'].apply(temp_category)

# 범주형 변수 인코딩
le_league = LabelEncoder()
le_home = LabelEncoder()
le_away = LabelEncoder()
le_stadium = LabelEncoder()
le_weather = LabelEncoder()

df_ml_fe['LEAGUE_ENCODED'] = le_league.fit_transform(df_ml_fe['LEAGUE_NAME'])
df_ml_fe['HomeTeam_ENCODED'] = le_home.fit_transform(df_ml_fe['HomeTeam'])
df_ml_fe['AwayTeam_ENCODED'] = le_away.fit_transform(df_ml_fe['AwayTeam'])
df_ml_fe['Stadium_ENCODED'] = le_stadium.fit_transform(df_ml_fe['Field_Name'])
df_ml_fe['Weather_ENCODED'] = le_weather.fit_transform(df_ml_fe['Weather_Simple'])

print("=== 특성 공학 완료 ===\n")
print(f"총 특성 수: {df_ml_fe.shape[1]}")
print("\n새로 생성된 특성:")
print("- Season_Stage: 시즌 단계 (0=초반, 1=중반, 2=후반)")
print("- Is_Weekend: 주말 여부 (0=평일, 1=주말)")
print("- Rank_Difference: 순위 차이 (홈-원정)")
print("- Top_Match: 상위권 대결 여부")
print("- Bottom_Match: 하위권 대결 여부")
print("- Weather_Simple: 날씨 단순화")
print("- Temp_Category: 온도 구간")
print("\n샘플 데이터:")
display(df_ml_fe[['HomeTeam', 'AwayTeam', 'HomeRank', 'AwayRank', 'Rank_Difference', 
                   'Is_Weekend', 'Season_Stage', 'Top_Match', 'Temp_Category', 'Audience_Qty']].head(10))

=== 특성 공학 완료 ===

총 특성 수: 26

새로 생성된 특성:
- Season_Stage: 시즌 단계 (0=초반, 1=중반, 2=후반)
- Is_Weekend: 주말 여부 (0=평일, 1=주말)
- Rank_Difference: 순위 차이 (홈-원정)
- Top_Match: 상위권 대결 여부
- Bottom_Match: 하위권 대결 여부
- Weather_Simple: 날씨 단순화
- Temp_Category: 온도 구간

샘플 데이터:


,HomeTeam,AwayTeam,HomeRank,AwayRank,Rank_Difference,Is_Weekend,Season_Stage,Top_Match,Temp_Category,Audience_Qty
0,인천,대전,9,2,7,1,0,0,0,10011
1,수원FC,포항,6,1,5,1,0,0,1,7155
2,대구,제주,8,6,2,1,0,0,1,10851
3,전북,수원,10,11,-1,1,0,0,1,19660
4,강원,울산,12,3,9,1,0,0,0,6199
5,광주,서울,5,3,2,1,0,1,1,7357
6,수원FC,수원,10,10,0,1,0,0,2,8670
7,대전,포항,4,1,3,1,0,1,2,8661
8,강원,대구,12,8,4,1,0,0,2,4058
9,서울,울산,2,3,-1,1,0,1,0,20549


# 2. 데이터 분할 및 정규화

In [14]:
# 특성 선택
feature_columns = [
    'LEAGUE_ENCODED',
    'Meet_Year',
    'Round',
    'HomeTeam_ENCODED',
    'AwayTeam_ENCODED',
    'HomeRank',
    'AwayRank',
    'Rank_Difference',
    'Stadium_ENCODED',
    'Temperature',
    'Humidity',
    'Weather_ENCODED',
    'Season_Stage',
    'Is_Weekend',
    'Top_Match',
    'Bottom_Match',
    'Temp_Category'
]

X = df_ml_fe[feature_columns]
y = df_ml_fe['Audience_Qty']

# 훈련/테스트 데이터 분할 (80:20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print("=== 데이터 분할 완료 ===\n")
print(f"전체 데이터: {len(X):,}건")
print(f"훈련 데이터: {len(X_train):,}건 ({len(X_train)/len(X)*100:.1f}%)")
print(f"테스트 데이터: {len(X_test):,}건 ({len(X_test)/len(X)*100:.1f}%)")
print(f"\n특성 개수: {X.shape[1]}개")
print(f"\n타겟 변수 (관중 수) 통계:")
print(f"  평균: {y.mean():.0f}명")
print(f"  표준편차: {y.std():.0f}명")
print(f"  최소: {y.min():,}명")
print(f"  최대: {y.max():,}명")

# 정규화 (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✓ 특성 정규화 완료 (StandardScaler)")

=== 데이터 분할 완료 ===

전체 데이터: 1,392건
훈련 데이터: 1,113건 (80.0%)
테스트 데이터: 279건 (20.0%)

특성 개수: 17개

타겟 변수 (관중 수) 통계:
  평균: 6841명
  표준편차: 6474명
  최소: 359명
  최대: 52,600명

✓ 특성 정규화 완료 (StandardScaler)


# 3. 여러 모델 학습 및 비교

In [15]:
# 여러 모델 정의
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=-1),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=5, random_state=42),
    'XGBoost': xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42, n_jobs=-1),
    'LightGBM': lgb.LGBMRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42, n_jobs=-1, verbose=-1)
}

# 모델 학습 및 평가
results = []

print("=== 모델 학습 시작 ===\n")

for name, model in models.items():
    print(f"학습 중: {name}...", end=" ")
    
    # Linear Regression은 정규화된 데이터 사용
    if name == 'Linear Regression':
        model.fit(X_train_scaled, y_train)
        y_pred_train = model.predict(X_train_scaled)
        y_pred_test = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred_train = model.predict(X_train)
        y_pred_test = model.predict(X_test)
    
    # 평가 지표 계산
    train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    train_mae = mean_absolute_error(y_train, y_pred_train)
    test_mae = mean_absolute_error(y_test, y_pred_test)
    train_r2 = r2_score(y_train, y_pred_train)
    test_r2 = r2_score(y_test, y_pred_test)
    
    results.append({
        'Model': name,
        'Train_RMSE': train_rmse,
        'Test_RMSE': test_rmse,
        'Train_MAE': train_mae,
        'Test_MAE': test_mae,
        'Train_R2': train_r2,
        'Test_R2': test_r2
    })
    
    print(f"완료!")

print("\n✓ 모든 모델 학습 완료\n")

# 결과를 DataFrame으로 변환
df_results = pd.DataFrame(results)
df_results = df_results.sort_values('Test_RMSE')

print("=== 모델 성능 비교 ===\n")
display(df_results.round(2))

=== 모델 학습 시작 ===

학습 중: Linear Regression... 완료!
학습 중: Random Forest... 완료!
학습 중: Gradient Boosting... 완료!
학습 중: XGBoost... 완료!
학습 중: LightGBM... 완료!

✓ 모든 모델 학습 완료

=== 모델 성능 비교 ===



,Model,Train_RMSE,Test_RMSE,Train_MAE,Test_MAE,Train_R2,Test_R2
4,LightGBM,1865.03,2633.05,1156.49,1749.17,0.92,0.80
3,XGBoost,1192.53,2681.59,783.96,1707.20,0.97,0.79
2,Gradient Boosting,1050.47,2737.08,733.50,1751.75,0.97,0.78
1,Random Forest,1245.45,2859.95,765.51,1951.10,0.96,0.76
0,Linear Regression,5282.30,4712.43,3422.73,3252.03,0.36,0.35


In [16]:
# Plotly 시각화: 모델 성능 비교 (RMSE, MAE, R²)
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('RMSE (낮을수록 좋음)', 'MAE (낮을수록 좋음)', 'R² (높을수록 좋음)'),
    horizontal_spacing=0.1
)

# RMSE 비교
fig.add_trace(
    go.Bar(
        x=df_results['Model'],
        y=df_results['Test_RMSE'],
        name='Test RMSE',
        marker_color='#FF6B6B',
        text=df_results['Test_RMSE'].round(0),
        textposition='auto'
    ),
    row=1, col=1
)

# MAE 비교
fig.add_trace(
    go.Bar(
        x=df_results['Model'],
        y=df_results['Test_MAE'],
        name='Test MAE',
        marker_color='#4ECDC4',
        text=df_results['Test_MAE'].round(0),
        textposition='auto'
    ),
    row=1, col=2
)

# R² 비교
fig.add_trace(
    go.Bar(
        x=df_results['Model'],
        y=df_results['Test_R2'],
        name='Test R²',
        marker_color='#95E1D3',
        text=df_results['Test_R2'].round(3),
        textposition='auto'
    ),
    row=1, col=3
)

fig.update_xaxes(tickangle=45)
fig.update_yaxes(title_text="RMSE (명)", row=1, col=1)
fig.update_yaxes(title_text="MAE (명)", row=1, col=2)
fig.update_yaxes(title_text="R² Score", row=1, col=3)

fig.update_layout(
    height=500,
    title_text="머신러닝 모델 성능 비교 (테스트 데이터)",
    showlegend=False
)

fig.show()

# 최고 성능 모델 선택
best_model_name = df_results.iloc[0]['Model']
print(f"\n🏆 최고 성능 모델: {best_model_name}")
print(f"  - Test RMSE: {df_results.iloc[0]['Test_RMSE']:.2f}명")
print(f"  - Test MAE: {df_results.iloc[0]['Test_MAE']:.2f}명")
print(f"  - Test R²: {df_results.iloc[0]['Test_R2']:.4f}")
print(f"\n해석: 평균적으로 실제 관중 수와 약 ±{df_results.iloc[0]['Test_MAE']:.0f}명 차이로 예측")


🏆 최고 성능 모델: LightGBM
  - Test RMSE: 2633.05명
  - Test MAE: 1749.17명
  - Test R²: 0.7969

해석: 평균적으로 실제 관중 수와 약 ±1749명 차이로 예측


# 4. 특성 중요도 분석 (Feature Importance)

In [17]:
# 최고 성능 모델로 특성 중요도 분석
best_model = models[best_model_name]

# 트리 기반 모델만 feature_importances_ 제공
if hasattr(best_model, 'feature_importances_'):
    # 특성 중요도 추출
    feature_importance = pd.DataFrame({
        'Feature': feature_columns,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    # 특성 이름을 더 읽기 쉽게 변환
    feature_name_map = {
        'LEAGUE_ENCODED': '리그',
        'Meet_Year': '연도',
        'Round': '라운드',
        'HomeTeam_ENCODED': '홈팀',
        'AwayTeam_ENCODED': '원정팀',
        'HomeRank': '홈팀 순위',
        'AwayRank': '원정팀 순위',
        'Rank_Difference': '순위 차이',
        'Stadium_ENCODED': '경기장',
        'Temperature': '온도',
        'Humidity': '습도',
        'Weather_ENCODED': '날씨',
        'Season_Stage': '시즌 단계',
        'Is_Weekend': '주말 여부',
        'Top_Match': '상위권 대결',
        'Bottom_Match': '하위권 대결',
        'Temp_Category': '온도 구간'
    }
    
    feature_importance['Feature_KR'] = feature_importance['Feature'].map(feature_name_map)
    
    print(f"=== {best_model_name} 특성 중요도 Top 10 ===\n")
    display(feature_importance.head(10))
    
    # Plotly 시각화: 특성 중요도
    fig = go.Figure()
    
    top_n = 15
    df_plot = feature_importance.head(top_n).sort_values('Importance', ascending=True)
    
    fig.add_trace(
        go.Bar(
            y=df_plot['Feature_KR'],
            x=df_plot['Importance'],
            orientation='h',
            marker=dict(
                color=df_plot['Importance'],
                colorscale='Viridis',
                showscale=True
            ),
            text=df_plot['Importance'].round(3),
            textposition='auto',
            hovertemplate='<b>%{y}</b><br>중요도: %{x:.4f}<extra></extra>'
        )
    )
    
    fig.update_layout(
        title=f'{best_model_name} 특성 중요도 (Top {top_n})',
        xaxis_title='중요도',
        yaxis_title='특성',
        height=600,
        showlegend=False
    )
    
    fig.show()
    
else:
    print(f"{best_model_name}은 특성 중요도를 제공하지 않습니다.")
    print("대신 Random Forest 모델의 특성 중요도를 출력합니다.\n")
    
    rf_model = models['Random Forest']
    feature_importance = pd.DataFrame({
        'Feature': feature_columns,
        'Importance': rf_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    feature_name_map = {
        'LEAGUE_ENCODED': '리그',
        'Meet_Year': '연도',
        'Round': '라운드',
        'HomeTeam_ENCODED': '홈팀',
        'AwayTeam_ENCODED': '원정팀',
        'HomeRank': '홈팀 순위',
        'AwayRank': '원정팀 순위',
        'Rank_Difference': '순위 차이',
        'Stadium_ENCODED': '경기장',
        'Temperature': '온도',
        'Humidity': '습도',
        'Weather_ENCODED': '날씨',
        'Season_Stage': '시즌 단계',
        'Is_Weekend': '주말 여부',
        'Top_Match': '상위권 대결',
        'Bottom_Match': '하위권 대결',
        'Temp_Category': '온도 구간'
    }
    
    feature_importance['Feature_KR'] = feature_importance['Feature'].map(feature_name_map)
    
    display(feature_importance.head(10))

=== LightGBM 특성 중요도 Top 10 ===



,Feature,Importance,Feature_KR
4,AwayTeam_ENCODED,219,원정팀
3,HomeTeam_ENCODED,217,홈팀
8,Stadium_ENCODED,182,경기장
2,Round,166,라운드
5,HomeRank,139,홈팀 순위
10,Humidity,138,습도
9,Temperature,134,온도
6,AwayRank,110,원정팀 순위
0,LEAGUE_ENCODED,85,리그
7,Rank_Difference,84,순위 차이


# 5. 예측 결과 시각화

In [18]:
# 최고 성능 모델로 예측
if best_model_name == 'Linear Regression':
    y_pred_final = best_model.predict(X_test_scaled)
else:
    y_pred_final = best_model.predict(X_test)

# 예측 결과 DataFrame 생성
df_predictions = pd.DataFrame({
    'Actual': y_test.values,
    'Predicted': y_pred_final,
    'Error': y_test.values - y_pred_final,
    'Error_Pct': (y_test.values - y_pred_final) / y_test.values * 100
})

print("=== 예측 결과 샘플 ===")
display(df_predictions.head(20))

print(f"\n예측 오차 통계:")
print(f"  평균 오차: {df_predictions['Error'].mean():.2f}명")
print(f"  평균 절대 오차: {df_predictions['Error'].abs().mean():.2f}명")
print(f"  평균 오차율: {df_predictions['Error_Pct'].abs().mean():.2f}%")

=== 예측 결과 샘플 ===


,Actual,Predicted,Error,Error_Pct
0,1274,2614.147644,-1340.147644,-105.192123
1,10620,12584.489661,-1964.489661,-18.498019
2,4828,4388.491154,439.508846,9.103332
3,2576,2406.628015,169.371985,6.574999
4,7220,12762.050974,-5542.050974,-76.759709
5,3871,2679.348237,1191.651763,30.784081
6,2019,2396.399244,-377.399244,-18.692385
7,6204,6984.170610,-780.170610,-12.575284
8,9203,9399.080683,-196.080683,-2.130617
9,2387,3300.881742,-913.881742,-38.285787



예측 오차 통계:
  평균 오차: -61.74명
  평균 절대 오차: 1749.17명
  평균 오차율: 37.47%


In [19]:
# Plotly 시각화: 실제 vs 예측 산점도
fig = go.Figure()

# 산점도
fig.add_trace(
    go.Scatter(
        x=df_predictions['Actual'],
        y=df_predictions['Predicted'],
        mode='markers',
        name='예측',
        marker=dict(
            size=6,
            color=df_predictions['Error'].abs(),
            colorscale='Reds',
            showscale=True,
            colorbar=dict(title='절대 오차'),
            opacity=0.6
        ),
        hovertemplate='<b>실제: %{x:,}명</b><br>예측: %{y:,}명<br>오차: %{customdata:,.0f}명<extra></extra>',
        customdata=df_predictions['Error']
    )
)

# 완벽한 예측선 (y=x)
min_val = min(df_predictions['Actual'].min(), df_predictions['Predicted'].min())
max_val = max(df_predictions['Actual'].max(), df_predictions['Predicted'].max())
fig.add_trace(
    go.Scatter(
        x=[min_val, max_val],
        y=[min_val, max_val],
        mode='lines',
        name='완벽한 예측',
        line=dict(color='red', width=2, dash='dash'),
        showlegend=True
    )
)

fig.update_layout(
    title=f'{best_model_name} - 실제 관중 vs 예측 관중',
    xaxis_title='실제 관중 수 (명)',
    yaxis_title='예측 관중 수 (명)',
    height=600,
    hovermode='closest'
)

fig.show()

In [20]:
# Plotly 시각화: 예측 오차 분포
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('예측 오차 분포 (명)', '예측 오차율 분포 (%)'),
    horizontal_spacing=0.12
)

# 오차 히스토그램 (명)
fig.add_trace(
    go.Histogram(
        x=df_predictions['Error'],
        nbinsx=50,
        name='오차 (명)',
        marker_color='#4ECDC4',
        hovertemplate='오차: %{x:.0f}명<br>빈도: %{y}<extra></extra>'
    ),
    row=1, col=1
)

# 오차율 히스토그램 (%)
fig.add_trace(
    go.Histogram(
        x=df_predictions['Error_Pct'],
        nbinsx=50,
        name='오차율 (%)',
        marker_color='#FF6B6B',
        hovertemplate='오차율: %{x:.1f}%<br>빈도: %{y}<extra></extra>'
    ),
    row=1, col=2
)

# 평균선 추가
fig.add_vline(x=df_predictions['Error'].mean(), line_dash="dash", line_color="red", 
                annotation_text=f"평균: {df_predictions['Error'].mean():.0f}명", row=1, col=1)
fig.add_vline(x=df_predictions['Error_Pct'].mean(), line_dash="dash", line_color="red",
                annotation_text=f"평균: {df_predictions['Error_Pct'].mean():.1f}%", row=1, col=2)

fig.update_xaxes(title_text="오차 (명)", row=1, col=1)
fig.update_xaxes(title_text="오차율 (%)", row=1, col=2)
fig.update_yaxes(title_text="빈도", row=1, col=1)
fig.update_yaxes(title_text="빈도", row=1, col=2)

fig.update_layout(
    title_text="예측 오차 분포 분석",
    showlegend=False,
    height=500
)

fig.show()

# 6. 머신러닝 모델 결론

## 모델 성능 요약
위에서 학습한 5가지 머신러닝 모델을 비교한 결과, 트리 기반 앙상블 모델들(Random Forest, XGBoost, LightGBM)이 선형 모델보다 우수한 성능을 보였습니다.

## 주요 인사이트

### 1. 관중 예측의 핵심 특성
특성 중요도 분석을 통해 다음 요소들이 관중 수 예측에 중요한 영향을 미치는 것으로 확인:
- **홈팀/경기장**: 팀 브랜드와 경기장 위치가 가장 큰 영향
- **원정팀**: 상대 팀의 인기도도 중요한 요소
- **주말 여부**: 경기 요일이 관중 동원에 큰 영향
- **순위 관련 특성**: 홈팀/원정팀 순위, 순위 차이
- **시즌 단계**: 시즌 초반/중반/후반에 따른 관중 패턴
- **날씨 요인**: 온도, 날씨 상태 (보조적 영향)

### 2. 모델의 한계
- **R² 값 해석**: 모델이 관중 수 변동의 일부만 설명 가능
- **예측하기 어려운 요인들**:
  - 팀의 최근 경기 결과 (연승/연패)
  - 특별 이벤트 (더비 매치, 특별 기념일)
  - 경쟁 스포츠 이벤트 (월드컵, 올림픽 등)
  - 사회적 이슈 (팬데믹, 경제 상황 등)
  - 티켓 가격 정책
  
### 3. 실무 활용 방안
- **관중 수 예측**: 경기 일정 수립 시 예상 관중 수 예측
- **마케팅 전략**: 관중이 적을 것으로 예상되는 경기의 프로모션 강화
- **자원 배치**: 예상 관중에 따른 경기장 운영 인력 및 시설 준비
- **수익 예측**: 티켓 판매 및 부가 수익 예상

### 4. 개선 방향
**추가 특성 확보 시 성능 향상 가능**:
- 팀의 최근 5경기 성적 (연승/연패 여부)
- 경기 중요도 (우승 경쟁, 강등 경쟁)
- 티켓 가격 정보
- 경쟁 이벤트 여부
- SNS 언급량, 관심도 지표
- Physical Metrics (경기 활동량 데이터)

## 결론
머신러닝 모델을 통해 K리그 관중 수를 어느 정도 예측할 수 있으며, **홈팀 브랜드, 경기장, 요일**이 가장 중요한 예측 변수임을 확인했습니다. 

이는 앞선 통계 분석 결과와 일치하며, **팀의 인기도와 경기 접근성(주말, 경기장 위치)이 관중 동원의 핵심 요소**임을 재확인했습니다.

승강제 확대를 통한 리그 유동성 증가가 관중 증대에 미치는 영향을 정량화하기 위해서는 **승격/강등 전후 데이터**와 **장기 시계열 분석**이 추가로 필요합니다.